In [2]:
import pandas as pd
import datetime
import numpy as np
import math
import zipfile

In [3]:
zippath = "201712-citibike-tripdata.csv.zip"

In [4]:
zf = zipfile.ZipFile('201712-citibike-tripdata.csv.zip') 

In [5]:
data = pd.read_csv(zf.open('201712-citibike-tripdata.csv'))

In [ ]:
data.dtypes()

In [6]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,256,2017-12-01 00:00:00,2017-12-01 00:04:17,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,262,Washington Park,40.691782,-73.973730,18858,Subscriber,1981.0,1
1,325,2017-12-01 00:00:17,2017-12-01 00:05:43,470,W 20 St & 8 Ave,40.743453,-74.000040,490,8 Ave & W 33 St,40.751551,-73.993934,19306,Subscriber,1992.0,1
2,967,2017-12-01 00:00:19,2017-12-01 00:16:26,347,Greenwich St & W Houston St,40.728846,-74.008591,504,1 Ave & E 16 St,40.732219,-73.981656,28250,Subscriber,1992.0,1
3,125,2017-12-01 00:00:20,2017-12-01 00:02:26,3077,Stagg St & Union Ave,40.708771,-73.950953,3454,Leonard St & Maujer St,40.710369,-73.947060,25834,Subscriber,1988.0,1
4,451,2017-12-01 00:00:28,2017-12-01 00:08:00,368,Carmine St & 6 Ave,40.730386,-74.002150,326,E 11 St & 1 Ave,40.729538,-73.984267,14769,Subscriber,1986.0,1


In [7]:
null_counts = data.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

birth year    39445
dtype: int64

In [8]:
data['birth year'] = data['birth year'].fillna(data['birth year'].mode()[0])

In [9]:
#Converting timedata in datetime
data["starttime"] = pd.to_datetime(data['starttime'])
data["stoptime"] = pd.to_datetime(data["stoptime"])
data["hour"] = data["starttime"].map(lambda x: x.hour)
data["day"] = data["starttime"].map(lambda x:x.day)

In [10]:
def haversine(lat1,lon1,lat2,lon2):
    #lat1, lon1 = origin
    #lat2, lon2 = destination
    radius = 6371 # km
    radius = 3959 #miles
    
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [11]:
data["distance"] = data.apply(lambda x:haversine(x["start station latitude"],x["start station longitude"],x["end station latitude"], x["end station longitude"]),axis=1)

In [12]:
def speed(dist,time):
    speed = dist/(time/(60*60))
    return speed

In [13]:
data["speed"] = data.apply(lambda x: speed(x["distance"],x["tripduration"]),axis=1)

In [14]:
def age(age):
    return (2017-int(age))

In [15]:
data["age"] = data.apply(lambda x : age(x["birth year"]),axis=1)

In [16]:
data["dayOfWeek"] = data["starttime"].apply(lambda x:x.weekday())

In [17]:
data["start_date"] = data["starttime"].map(lambda x: x.day)
data["end_date"] = data["stoptime"].map(lambda x: x.day)

In [18]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,birth year,gender,hour,day,distance,speed,age,dayOfWeek,start_date,end_date
0,256,2017-12-01 00:00:00,2017-12-01 00:04:17,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,262,Washington Park,40.691782,...,1981.0,1,0,1,0.403405,5.672882,36,4,1,1
1,325,2017-12-01 00:00:17,2017-12-01 00:05:43,470,W 20 St & 8 Ave,40.743453,-74.000040,490,8 Ave & W 33 St,40.751551,...,1992.0,1,0,1,0.644398,7.137951,25,4,1,1
2,967,2017-12-01 00:00:19,2017-12-01 00:16:26,347,Greenwich St & W Houston St,40.728846,-74.008591,504,1 Ave & E 16 St,40.732219,...,1992.0,1,0,1,1.429495,5.321800,25,4,1,1
3,125,2017-12-01 00:00:20,2017-12-01 00:02:26,3077,Stagg St & Union Ave,40.708771,-73.950953,3454,Leonard St & Maujer St,40.710369,...,1988.0,1,0,1,0.231877,6.678067,29,4,1,1
4,451,2017-12-01 00:00:28,2017-12-01 00:08:00,368,Carmine St & 6 Ave,40.730386,-74.002150,326,E 11 St & 1 Ave,40.729538,...,1986.0,1,0,1,0.938194,7.488910,31,4,1,1


In [19]:
data.to_csv("201712-citibike-tripdata_cleaned.csv", encoding='utf-8',index=False)

In [20]:
data.dtypes()

TypeError: 'Series' object is not callable